In [1]:
import pandas as pd
import json
import numpy as np
import math
import heapq

# total_link = pd.read_csv("C:/Users/gimm7/OneDrive/바탕 화면/last_weight_arranegment.csv", encoding='cp949')
total_link = pd.read_csv("C:/Users/gimm7/OneDrive/바탕 화면/last_weight_arranegment.csv")
total_link.reset_index()
total_node = pd.read_csv("C:/capstone_design2_data/total_node.csv")

def compose_coordinate(latitude, longitude):
    composed_coordinate = '['+ str(longitude)+ ','+ str(latitude)+ ']'
    return composed_coordinate

# def haversine_distance(coord1, coord2):
#     R = 6371.0  # Earth's radius in km
#     lat1 = math.radians(coord1[0])
#     lon1 = math.radians(coord1[1])
#     lat2 = math.radians(coord2[0])
#     lon2 = math.radians(coord2[1])
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

#     distance = R * c
#     return distance

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c

    #distance = ((lat1-lat2)**2+(lon1-lon2)**2)**0.5



    return distance


In [2]:
total_node['경도반올림'] = total_node['경도'].round(9)
total_node['위도반올림'] = total_node['위도'].round(9)
total_node.drop_duplicates(keep='first')
total_node.duplicated(['경도반올림','위도반올림']).sum()
total_node.duplicated(['경도','위도']).sum()

108609

# 시작점 입력

In [3]:
dep_lat, dep_lon = 37.2825468,127.0382127
des_lat, des_lon = 37.2670259,127.0548853
# dep_lat = 37.28293695757347
# dep_lon = 127.0434230405271
# des_lat = 37.28312042096528
# des_lon = 127.04739397575078

# Get Nearest Node

In [4]:
def get_Nearest_Node(latitude, longitude):
    length_filter = 0.0003
    min_distance = 1e9
    nearest_lat = 0
    nearest_lon = 0
    nearest_idx = 0
    
    filtered_df = total_node.loc[(total_node['위도'] > latitude - length_filter) & (total_node['위도'] < latitude + length_filter) & (total_node['경도'] > longitude - length_filter) & (total_node['경도'] < longitude + length_filter)]
    for index, row in filtered_df.iterrows():

        cur_node_lat = row['위도']
        cur_node_lon = row['경도']
        cur_distance = haversine_distance(latitude, longitude, cur_node_lat, cur_node_lon)

        if cur_distance < min_distance:
            min_distance = cur_distance
            nearest_lat = cur_node_lat
            nearest_lon = cur_node_lon
            nearest_idx = index

    #return filtered_df
    return nearest_idx, nearest_lat, nearest_lon
    

start_idx, start_lat, start_lon = get_Nearest_Node(dep_lat, dep_lon)
end_idx, end_lat, end_lon = get_Nearest_Node(des_lat, des_lon)

In [5]:
print(start_idx, end_idx)

208571 202277


# 다익스트라 알고리즘

In [9]:

area_N = max(dep_lat, des_lat) + 0.005
area_S = min(dep_lat, des_lat) - 0.005
area_E = max(dep_lon, des_lon) + 0.005
area_W = min(dep_lon, des_lon) - 0.005

areal_link = total_link.loc[(total_link['노드1위도'] < area_N)
                             & (total_link['노드2위도'] < area_N)
                             & (total_link['노드1위도'] > area_S) 
                             & (total_link['노드2위도'] > area_S)
                             & (total_link['노드1경도'] < area_E)
                             & (total_link['노드2경도'] < area_E)
                             & (total_link['노드1경도'] > area_W)
                             & (total_link['노드2경도'] > area_W)]

In [10]:
num_rows = areal_link.shape[0]
INF = int(1e9) #최댓값 지정

graph = {}
# distance = [INF]*(num_rows+1)
# route = [-1]*(num_rows+1)
distance = {}
route = {}

for i in range(0, num_rows):
    row = areal_link.iloc[i,:]
    if int(row['idx노드1']) not in distance:
        distance.update({int(row['idx노드1']):INF})
        route.update({int(row['idx노드1']):-1})
    
    if int(row['idx노드2']) not in distance:
        distance.update({int(row['idx노드2']):INF})
        route.update({int(row['idx노드2']):-1})


    if int(row['idx노드1']) in graph:
        graph[int(row['idx노드1'])].append((int(row['idx노드2']), row['last_weight']))
    else:
        graph.update({int(row['idx노드1']):[(int(row['idx노드2']), row['last_weight'])]})
    
    
    if int(row['idx노드2']) in graph:
        graph[int(row['idx노드2'])].append((int(row['idx노드1']), row['last_weight']))
    else:
        graph.update({int(row['idx노드2']):[(int(row['idx노드1']), row['last_weight'])]})


def dijkstra(start):
    q = []
    heapq.heappush(q,(0,start))
    distance[start] = 0
    route[start] = 0

    while q:
        dist, node = heapq.heappop(q) #우선순위 큐(heapq)를 사용
        if distance[node] < dist: #현재 저장된 node까지의 거리가 q에서 꺼낸 거리(dist)보다 작은 경우에는 continue
             continue
        for next in graph[node]: 
            cost = dist + next[1] 
            if cost < distance[next[0]]:
                distance[next[0]] = cost
                route[next[0]] = node 
                heapq.heappush(q,(cost,next[0]))
    return

dijkstra(start_idx)

def route_find(route, end_idx):
    route_list = []
    present = end_idx
    while(True):
        route_list.append(present)
        present = route[present]
        if present == 0:
            route_list.append(start_idx)
            break
    return route_list

route_list = route_find(route, end_idx)
print(route_list)

[202277, 202276, 201459, 201456, 201457, 202269, 202270, 202271, 202272, 202273, 201794, 201618, 201612, 201613, 201614, 201615, 201616, 201617, 201662, 201663, 201089, 201090, 201091, 201092, 201093, 201094, 205653, 205650, 205651, 205652, 205208, 205205, 205206, 203881, 203882, 205213, 205210, 203879, 203880, 205262, 205263, 203859, 203860, 205064, 205065, 203931, 203932, 373951, 375654, 375653, 373916, 373915, 375749, 375748, 203934, 203953, 205005, 205004, 205008, 204014, 204013, 205328, 205327, 203901, 197826, 197825, 221090, 221092, 221094, 221096, 220729, 220730, 220731, 220732, 84580, 84581, 84582, 84576, 84577, 84557, 84558, 84566, 84564, 84562, 84559, 84560, 84555, 84556, 205734, 205733, 201157, 201158, 201159, 201160, 208813, 207886, 207887, 208817, 207884, 207885, 385780, 385781, 382735, 382736, 386316, 382561, 382562, 384886, 384936, 384935, 382664, 382663, 386031, 386030, 386028, 384909, 384908, 385036, 385034, 207892, 208737, 208739, 207924, 207925, 207926, 208602, 21721

In [11]:
final_route = []
final_route.append([des_lat, des_lon])

for i in route_list:
    lat = total_node.iloc[i]["위도"]
    lon = total_node.iloc[i]["경도"]
    final_route.append([lat,lon])
final_route.append([dep_lat, dep_lon])

print(final_route)

[[37.2670259, 127.0548853], [37.26692507181171, 127.05498143355692], [37.26703464763264, 127.05519550558918], [37.26705382216251, 127.05523390871969], [37.2671460050456, 127.0554112369806], [37.26720236601689, 127.05518096008174], [37.267444896041816, 127.05418999979064], [37.267497878319176, 127.05397081200182], [37.26753309159701, 127.05383333790552], [37.267616620602666, 127.05348914297224], [37.26771663773516, 127.05306540825084], [37.26775811090566, 127.05288970713654], [37.26777557810375, 127.05281571576202], [37.267795926987766, 127.05272954901936], [37.26780129800692, 127.052706779668], [37.267851308943854, 127.05249490689124], [37.26792040995484, 127.05221172364116], [37.26793221640654, 127.05216167607357], [37.26799988582263, 127.05189485014972], [37.268048861176936, 127.05170172294496], [37.26806981054197, 127.0516191431403], [37.26809659412632, 127.05150479745794], [37.26811448577962, 127.05142839155071], [37.26841634725912, 127.05013863273868], [37.26845166329345, 127.0499

# Folium 테스트

In [12]:
import folium
import math


def makeMap(dep_lat, dep_lon, des_lat, des_lon):

    euclid_dist = math.sqrt((des_lat - dep_lat)**2 + (des_lon - dep_lon)**2)
    print(euclid_dist)


    map_osm = folium.Map(location=[(dep_lat + des_lat)/2, (dep_lon + des_lon)/2], zoom_start=17)
    # for i in final_route:
    #     folium.Marker(location=[i[0], i[1]]).add_to(map_osm)

    # Define coordinates for a line
    line_coordinates = final_route
    # Create a PolyLine object with the line coordinates and add it to the map
    folium.PolyLine(locations=line_coordinates, color='blue').add_to(map_osm)




    map_osm.save('C:/Users/gimm7/OneDrive/바탕 화면/map.html') #파일이 저장될 위치
    print("saved")

makeMap(dep_lat, dep_lon, des_lat, des_lon)

0.022778804348993098
saved
